# ** Описание **

In [110]:
#Точно второй питон
print 2
for i in xrange(4):
    print i

2
0
1
2
3


In [60]:
from __future__ import division
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import CountVectorizer

import base64
import csv
import gzip
import zlib

from collections import namedtuple
import numpy as np
import scipy
from scipy.sparse import csr_matrix

In [3]:
TRACE_NUM = 1000
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

def trace(items_num, trace_num=TRACE_NUM):
    if items_num % trace_num == 0: logging.info("Complete items %05d" % items_num)

### Утилиты

#### Декораторы

In [4]:
def to_utf8(text):
    if isinstance(text, unicode): text = text.encode('utf8')
    return text

def convert2unicode(f):
    def tmp(text):
        if not isinstance(text, unicode): text = text.decode('utf8')
        return f(text)
    return tmp

def convert2lower(f):
    def tmp(text):        
        return f(text.lower())
    return tmp

#P.S. Декораторы могут усложнять отладку, так что от них вполне можно отказаться и воспользоваться copy-paste

### Извлечение текста из html

#### Извлечение текста при помощи встроенных модулей

In [5]:
from HTMLParser import HTMLParser
import re

###Извлечение текста из title можно вписать сюда

class TextHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._text = []
        self._title = ""
        self._in_title = False

    def handle_data(self, data):
        text = data.strip()
        if len(text) > 0:
            text = re.sub('[ \t\r\n]+', ' ', text)
            self._text.append(text + ' ')
            if self._in_title == True:
                self._title = text

    def handle_starttag(self, tag, attrs):
        if tag == 'p':
            self._text.append('\n\n')
        elif tag == 'br':
            self._text.append('\n')
        elif tag == 'title':
            self._in_title = True

    def handle_startendtag(self, tag, attrs):
        if tag == 'br':
            self._text.append('\n\n')
    def handle_endtag(self, tag):
        if tag == 'title':
            self._in_title = False

    def text(self):
        return ''.join(self._text).strip()
    def title(self):
        return ''.join(self._title).strip()

@convert2unicode
def html2text_parser(text):
    parser = TextHTMLParser()
    parser.feed(text)
    return parser

#### Извлечение текста при помощи дополнительных библиотек

In [6]:
def html2text_bs(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста
    """
    soup = BeautifulSoup(raw_html, "html.parser")
    [s.extract() for s in soup(['script', 'style'])]
    return soup.get_text()

def html2text_bs_visible(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста, который видим пользователю
    """
    soup = BeautifulSoup(raw_html, "html.parser")    
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    return soup.get_text()

def html2text_boilerpipe(raw_html):
    import boilerpipe
    """
    еще одна библиотека очень хорошо извлекающая именно видимый пользователю текст,
    но она завязана на java
    """
    pass

#### Выбираем какой метод для конвертации html в текст будет основным

In [7]:
#html2text = html2text_bs
html2text = html2text_parser

#### Методы для токенизации текста

In [8]:
@convert2lower
@convert2unicode
def easy_tokenizer(text):
    word = unicode()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = unicode()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
#hint, чтобы установка pymorphy2 не была бы обязательной
def get_lemmatizer():
    import pymorphy2
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

@convert2lower
@convert2unicode
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = get_lemmatizer().parse(word)[0].normal_form            
        yield PYMORPHY_CACHE[word_hash]

#### Основная функция, которая вызывается для преобразования html в список слов

In [9]:
def html2word(raw_html, to_text=html2text, tokenizer=easy_tokenizer):
    parser = to_text(raw_html)
    txt = parser.text()
    return tokenizer(txt.lower())

#### Рассчет финальных метрик

In [10]:
def safe_divide(a, b):
    if a == 0: return 0.0
    elif b == 0: return 0.0
    else: return a/b

def calculate_metrics(predictions, threshold):    
    """
    Функция подсчета метрик
    Параметры
    predictions - ранки по документам
    threshold  - порог для метрик
    """
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    for (url_id, mark, url, prediction) in predictions:        
        mark_predict = prediction > threshold

        if mark_predict:                     
            if mark_predict == mark: true_positive += 1
            else: false_positive += 1                    
        else:                     
            if  mark_predict == mark: true_negative += 1
            else: false_negative += 1

    class_prec  = safe_divide(true_positive, true_positive + false_positive)
    class_recall = safe_divide(true_positive, true_positive + false_negative)
        
    class_F1 = safe_divide(2 * class_prec * class_recall, class_prec + class_recall)
    
    
    not_class_prec = safe_divide(true_negative, true_negative + false_negative)
    not_class_recall = safe_divide(true_negative, true_negative + false_positive)
    
    not_class_F1 = safe_divide(2 * not_class_prec * not_class_recall, not_class_prec + not_class_recall)
    
    return ( (class_prec, class_recall, class_F1), (not_class_prec, not_class_recall, not_class_F1) )

def arange(start, stop, step):
    cur_value = start
    while True:
        if cur_value > stop: break
        yield cur_value
        cur_value += step

def plot_results(docs, min_threshold=-1, max_threshold=1, step=0.1, trace=False):
    x = []
    y_p = []
    y_n = []
    docs_predictions = classifier.predict_all(docs)
    for threshold in arange(min_threshold, max_threshold, step):
        r = calculate_metrics(docs_predictions, threshold)
        x.append(threshold)
        y_p.append(r[0])
        y_n.append(r[1])        
        if trace: 
            print 'threshold %s' % threshold
            print '\tclass_prec %s, class_recall %s, class_F1 %s' % r[0]
            print '\tnot_class_prec %s, not_class_recall %s, not_class_F1 %s' % r[1]
            print '\t\tMacroF1Mesure %s' % ((r[0][2] + r[1][2])/2)
    plot_stats(x, y_p, "Class Result")
    plot_stats(x, y_n, "Not class Result")    


def plot_stats(x, y, title):
    plt.figure(figsize=(10, 5))

    prec, = plt.plot( x, 
                     [k[0] for k in y], "r", label='Precision', 
                     linewidth=1)
    accur, = plt.plot( x, 
                      [k[1] for k in y], "b", label='Recall',
                      linewidth=1)
    f1, =    plt.plot( x, 
                      [k[2] for k in y], "g", label='F1',
                      linewidth=1)
    plt.grid(True)
    plt.legend(handles=[prec, accur, f1])
    plt.title(title)
    plt.show()

In [112]:
DocItem = namedtuple('DocItem', ['doc_id', 'is_spam', 'url', 'features'])


def load_csv(input_file_name):    
    """
    Загружаем данные и извлекаем на лету признаки
    Сам контент не сохраняется, чтобы уменьшить потребление памяти - чтобы
    можно было запускать даже на ноутбуках в классе
    """
    
    with gzip.open(input_file_name) if input_file_name.endswith('gz') else open(input_file_name)  as input_file:            
        headers = input_file.readline()
        
        for i, line in enumerate(input_file):
            trace(i)
            parts = line.strip().split('\t')
            url_id = int(parts[0])                                        
            mark = bool(int(parts[1]))                    
            url = parts[2]
            pageInb64 = parts[3]
            html_data = base64.b64decode(pageInb64)
            yield url_id, html2word(html_data), mark
       
                
        trace(i, 1)        

In [113]:
DocItem = namedtuple('DocItem', ['doc_id', 'is_spam', 'url', 'features'])

#Это вспомогательная функция, с помощью который мы хотим получить векторное представление слов.
def load_words(input_file_name):    
    
    with gzip.open(input_file_name) if input_file_name.endswith('gz') else open(input_file_name)  as input_file:            
        headers = input_file.readline()
        
        for i, line in enumerate(input_file):
            trace(i)
            parts = line.strip().split('\t')
            html_data = base64.b64decode(parts[3])
            spisok = html2word(html_data)
            for w in spisok:
                yield w         
                
        trace(i, 1)        

** Обрабатываем входной файл **
<br>
Формат - поля разделенные табуляциями
<br>
0 - идентификатор документа
<br>
1 - метка класса 0 - не спам, 1 - спам
<br>
2 - урл документа
<br>
3 - документ в кодировке base64

Выходной формат - массив кортежей вида
(doc_id, is_spam, url, html_data)

In [49]:
cv = CountVectorizer()
TRAIN_DATA_FILE  = 'kaggle_train_data_tab.csv.gz'

In [142]:
#Так можно итерироваться по этим html, соответственно преобразовываем их в вектор с помощью обученного count_vectorizer.
for i, x in enumerate(load_csv(TRAIN_DATA_FILE)):
    x_lst = list(x[1])
    print i,'\n\n\n\n\n', x_lst
    if i > 2:
        break

23:11:24 INFO:Complete items 00000


0 




[u'\u0434\u043e\u0433\u043e\u0432\u043e\u0440', u'\u0437\u0430\u0439\u043c\u0430', u'\u0434\u043e\u0433\u043e\u0432\u043e\u0440\u044b', u'\u0434\u043e\u0433\u043e\u0432\u043e\u0440\u0430', u'function', u'poll', u'answer', u'question', u'adr', u'poll', u'php', u'a', u'answer', u'q', u'question', u't', u'toolbar', u'no', u'location', u'no', u'directories', u'no', u'status', u'no', u'menubar', u'no', u'scrollbars', u'no', u'resizable', u'no', u'width', u'300', u'height', u'100', u'window', u'open', u'adr', u't', u'function', u'photo', u'adr', u'adr', u'pictures', u'php', u'a', u'adr', u't', u'toolbar', u'no', u'location', u'no', u'directories', u'no', u'status', u'no', u'menubar', u'no', u'scrollbars', u'no', u'resizable', u'no', u'width', u'300', u'height', u'300', u'window', u'open', u'adr', u't', u'function', u'photo2', u'adr', u't', u'toolbar', u'no', u'location', u'no', u'directories', u'no', u'status', u'no', u'menubar', u'no', u'scrollbars', u'no', u'resizable', u'yes', u'wi

In [90]:
#Для начала нужно получить большой-большой спарс-вектор.
start_get_dict = load_words(TRAIN_DATA_FILE)
%time
cv.fit(start_get_dict)

#Теперь нужно привести датасет в матричный вид и проставить метки.
data = []
labels = []
id_doc = []
for i, x in enumerate(load_csv(TRAIN_DATA_FILE)):
    #print (x[0], list(x[1]), x[2])
    transformed = cv.transform(list(x[1]))
    #print cv.get_feature_names()
    #break
    cs_matr = csr_matrix(transformed.sum(axis=0))
    data.append(cs_matr)
    labels.append(x[2])
    id_doc.append(x[0])

21:56:56 INFO:Complete items 00000


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


21:57:16 INFO:Complete items 01000
21:57:33 INFO:Complete items 02000
21:57:50 INFO:Complete items 03000
21:58:07 INFO:Complete items 04000
21:58:25 INFO:Complete items 05000
21:58:41 INFO:Complete items 06000
21:58:59 INFO:Complete items 07000
21:59:00 INFO:Complete items 07043
21:59:06 INFO:Complete items 00000
21:59:31 INFO:Complete items 01000
21:59:51 INFO:Complete items 02000
22:00:13 INFO:Complete items 03000
22:00:37 INFO:Complete items 04000
22:01:01 INFO:Complete items 05000
22:01:22 INFO:Complete items 06000
22:01:45 INFO:Complete items 07000
22:01:46 INFO:Complete items 07043


Функция трейн и предикт олл написаны вручную, так как так удобнее с ними работать(потому что словарь, который мы получили на первом шаге, уже готов, и его можно непосредственно юзать для обучения и предсказания).

In [105]:
#Стакаем теперь все вместе.
#Юзаем дефолтный РэндомФорестКлассифайер
from sklearn.ensemble import RandomForestClassifier
#глубина дерева - очень важно, когда я ограничил ее до 3, качество было не очень
model = RandomForestClassifier(n_estimators = 50)
X = scipy.sparse.vstack(data, format='csr')
y = np.array(labels)
model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [106]:
import pandas as pd
TEST_DATA_FILE  = 'kaggle_test_data_tab.csv.gz'


final_pd = pd.DataFrame()
index = []
prediction = []
for i, x in enumerate(load_csv(TEST_DATA_FILE)):
    transformed = cv.transform(list(x[1]))
    #print transformed
    #print cv.get_feature_names()
    #break
    final_test = csr_matrix(transformed.sum(axis=0))
    predicted = model.predict(final_test)[0]

    
    index.append(x[0])
    prediction.append(int(predicted))
    #print index, prediction
    
final_pd['Id'] = index
final_pd['Prediction'] = prediction
 

22:32:29 INFO:Complete items 00000
22:33:17 INFO:Complete items 01000
22:34:01 INFO:Complete items 02000
22:34:45 INFO:Complete items 03000
22:35:26 INFO:Complete items 04000
22:36:07 INFO:Complete items 05000
22:36:51 INFO:Complete items 06000
22:37:32 INFO:Complete items 07000
22:38:15 INFO:Complete items 08000
22:38:57 INFO:Complete items 09000
22:39:44 INFO:Complete items 10000
22:40:25 INFO:Complete items 11000
22:41:07 INFO:Complete items 12000
22:41:47 INFO:Complete items 13000
22:42:31 INFO:Complete items 14000
22:43:15 INFO:Complete items 15000
22:43:58 INFO:Complete items 16000
22:44:00 INFO:Complete items 16038


In [107]:
final_pd.head()
final_pd.to_csv('result.csv', index = False)

In [108]:
final_pd.head()

,Id,Prediction
0,-9219333997282093076,1
1,-9216796593526314243,1
2,-9216634615606576667,0
3,-9215626373068452590,0
4,-9215403080479774736,1


In [126]:
transformed.toarray().shape

(869, 632671)

In [120]:
a = np.array([[1,1],[2,2]])

In [121]:
a.sum(axis = 0)

array([3, 3])

In [133]:
start_get_dict = load_words(TRAIN_DATA_FILE)
%time
cv.fit(start_get_dict)

23:06:10 INFO:Complete items 00000


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 14.1 µs


23:06:30 INFO:Complete items 01000
23:06:45 INFO:Complete items 02000
23:07:02 INFO:Complete items 03000
23:07:18 INFO:Complete items 04000
23:07:36 INFO:Complete items 05000
23:07:51 INFO:Complete items 06000
23:08:10 INFO:Complete items 07000
23:08:10 INFO:Complete items 07043


CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [148]:
for i, x in enumerate(load_csv(TRAIN_DATA_FILE)):
    #print (x[0], list(x[1]), x[2])
    #transformed = cv.transform(list(x[1]))
    t = list(x[1])
    break

23:12:56 INFO:Complete items 00000


In [149]:
#Разная длина, поэтому и получается матрицв
for x in t:
    print len(x)

7
5
8
8
8
4
6
8
3
4
3
1
6
1
8
1
7
2
8
2
11
2
6
2
7
2
10
2
9
2
5
3
6
3
6
4
3
1
8
5
3
3
8
3
1
3
1
7
2
8
2
11
2
6
2
7
2
10
2
9
2
5
3
6
3
6
4
3
1
8
6
3
1
7
2
8
2
11
2
6
2
7
2
10
2
9
3
5
4
6
3
6
4
3
1
8
13
6
2
6
9
12
6
9
16
8
12
6
2
6
9
16
6
9
12
5
3
6
10
6
4
5
1
8
8
5
3
10
6
6
1
9
1
3
5
5
6
7
2
5
6
4
2
5
6
6
4
7
7
4
6
2
6
6
6
4
4
1
6
6
4
10
6
6
5
1
6
6
5
5
6
6
6
3
5
6
6
6
3
5
6
6
6
4
5
6
6
6
5
5
6
6
6
6
7
5
8
5
2
4
4
4
10
3
4
2
6
2
6
7
2
2
4
7
1
8
6
6
8
7
11
8
7
1
6
5
6
4
1
6
7
18
6
6
8
6
8
4
2
7
5
5
1
4
10
7
5
7
7
4
7
7
7
5
1
12
10
7
6
7
5
6
9
7
14
11
7
5
7
7
7
8
7
14
3
7
9
10
7
5
7
5
5
12
6
7
5
5
11
6
4
5
11
5
10
1
10
5
7
4
5
10
5
10
1
11
5
7
7
2
8
14
5
8
14
10
12
8
1
10
12
7
11
8
5
5
1
4
7
1
8
3
1
13
11
12
8
2
12
15
7
10
1
18
14
5
8
12
9
11
8
9
12
10
11
12
8
8
8
16
10
2
10
1
10
8
5
6
12
1
9
8
8
5
7
5
7
5
1
10
16
12
7
12
7
7
13
6
14
1
11
7
4
8
5
7
1
11
5
3
4
5
12
8
10
2
13
6
4
10
1
13
7
8
3
7
8
6
2
4
5
1
3
2
10
1
4
2
8
2
6
8
1
11
2
7
8
4
5
15
3
3
8
3
1
3
7
11
8
5
7
5
6
8
6
12
1
1
7
10
4
